# Context-Aware Recommendation System


## Import Required Libraries


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Load the Dataset


In [2]:
df = pd.read_json("./res/export.json")

In [3]:
df

,wikipedia_ID,freebase_ID,title,author,pub_date,genres,summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,roman-à-clef satire childrens-literature specu...,"Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,science-fiction novella speculative-fiction ut...,"Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,existentialism fiction absurdist-fiction novel,The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,None,None,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,None,hard-science-fiction science-fiction speculati...,The novel posits that space around the Milky ...
...,...,...,...,...,...,...,...
16554,36934824,/m/0m0p0hr,Under Wildwood,Colin Meloy,2012-09-25,None,"Prue McKeel, having rescued her brother from ..."
16555,37054020,/m/04f1nbs,Transfer of Power,Vince Flynn,2000-06-01,thriller fiction,The reader first meets Rapp while he is doing...
16556,37122323,/m/0n5236t,Decoded,Jay-Z,2010-11-16,autobiography,The book follows very rough chronological ord...
16557,37132319,/m/0n4bqb1,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,2012-10-02,None,Colbert addresses topics including Wall Stree...


## Prepare the Dataset for ML


In [4]:
df.drop(columns=["wikipedia_ID", "freebase_ID", "pub_date"], axis=0, inplace=True)

In [5]:
df

,title,author,genres,summary
0,Animal Farm,George Orwell,roman-à-clef satire childrens-literature specu...,"Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,Anthony Burgess,science-fiction novella speculative-fiction ut...,"Alex, a teenager living in near-future Englan..."
2,The Plague,Albert Camus,existentialism fiction absurdist-fiction novel,The text of The Plague is divided into five p...
3,An Enquiry Concerning Human Understanding,David Hume,None,The argument of the Enquiry proceeds by a ser...
4,A Fire Upon the Deep,Vernor Vinge,hard-science-fiction science-fiction speculati...,The novel posits that space around the Milky ...
...,...,...,...,...
16554,Under Wildwood,Colin Meloy,None,"Prue McKeel, having rescued her brother from ..."
16555,Transfer of Power,Vince Flynn,thriller fiction,The reader first meets Rapp while he is doing...
16556,Decoded,Jay-Z,autobiography,The book follows very rough chronological ord...
16557,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,None,Colbert addresses topics including Wall Stree...


In [6]:
df.dropna(axis=0, how="any", inplace=True)  # Remove rows with missing values
df.reset_index(drop=True, inplace=True)  # Reset index
df

,title,author,genres,summary
0,Animal Farm,George Orwell,roman-à-clef satire childrens-literature specu...,"Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,Anthony Burgess,science-fiction novella speculative-fiction ut...,"Alex, a teenager living in near-future Englan..."
2,The Plague,Albert Camus,existentialism fiction absurdist-fiction novel,The text of The Plague is divided into five p...
3,A Fire Upon the Deep,Vernor Vinge,hard-science-fiction science-fiction speculati...,The novel posits that space around the Milky ...
4,All Quiet on the Western Front,Erich Maria Remarque,war-novel roman-à-clef,"The book tells the story of Paul Bäumer, a Ge..."
...,...,...,...,...
12050,The Third Lynx,Timothy Zahn,science-fiction,The story starts with former government agent...
12051,Remote Control,Andy McNab,thriller fiction suspense,The series follows the character of Nick Ston...
12052,Transfer of Power,Vince Flynn,thriller fiction,The reader first meets Rapp while he is doing...
12053,Decoded,Jay-Z,autobiography,The book follows very rough chronological ord...


## Recsys based on Genres


In [7]:
# Create a TF-IDF matrix
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix_genres = tfidf.fit_transform(df["genres"])

In [8]:
# Compute the cosine similarity matrix
cosine_sim_genres = cosine_similarity(tfidf_matrix_genres, tfidf_matrix_genres)

In [9]:
# Define a function to get book recommendations based on the cosine similarity score
def get_recs_genres(title, cosine_sim=cosine_sim_genres, df=df):
    # Get the index of the book that matches the title
    idx = df[df["title"] == title].index[0]

    # Get the pairwise similarity scores of all df with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the df based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top 100 most similar df
    sim_scores = sim_scores[0:100]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top 100 most similar df
    return df[:].iloc[book_indices]

In [10]:
left = pd.DataFrame(get_recs_genres("Glory Road"))
left

,title,author,genres,summary
73,The Wanderer,Fritz Leiber,science-fiction speculative-fiction fantasy fi...,The novel is set in a future a few decades ah...
136,Glory Road,Robert A. Heinlein,science-fiction speculative-fiction fantasy fi...,"Evelyn Cyril ""E.C."" Gordon (also known as ""Ea..."
285,Restoree,Anne McCaffrey,science-fiction speculative-fiction fantasy fi...,"Restoree is the story of Sara, an introverted..."
428,Stardust,Neil Gaiman,science-fiction speculative-fiction fantasy fi...,The Faerie Market is held every nine years on...
635,The Gods Themselves,Isaac Asimov,science-fiction speculative-fiction fantasy fi...,The main plotline is a project by aliens who ...
...,...,...,...,...
3166,Iron Fist,Aaron Allston,science-fiction speculative-fiction fantasy fi...,After Wraith Squadron's first successful miss...
3167,Solo Command,Aaron Allston,science-fiction speculative-fiction fantasy fi...,Wraith Squadron is once again tasked with des...
3170,The Hallowed Hunt,Lois McMaster Bujold,science-fiction speculative-fiction fantasy fi...,"The principal characters are: Ingrey, who rec..."
3242,Behold the Man,Michael Moorcock,science-fiction speculative-fiction fantasy fi...,The story begins with Karl's violent arrival ...


## Recsys Based on Description


In [11]:
# Create a TF-IDF matrix
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix_desc = tfidf.fit_transform(df["summary"])

In [12]:
# Compute the cosine similarity matrix
cosine_sim_desc = cosine_similarity(tfidf_matrix_desc, tfidf_matrix_desc)

In [13]:
# Define a function to get book recommendations based on the cosine similarity score
def get_recs_desc(title, cosine_sim=cosine_sim_desc, df=df):
    # Get the index of the book that matches the title
    idx = df[df["title"] == title].index[0]

    # Get the pairwise similarity scores of all df with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the df based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top 100 most similar df
    sim_scores = sim_scores[1:101]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top 100 most similar df
    return df[:].iloc[book_indices]

In [14]:
right = pd.DataFrame(get_recs_desc("Glory Road"))
right

,title,author,genres,summary
8755,The Brief Wondrous Life of Oscar Wao,Junot Díaz,fiction novel,This novel opens with an introductory section...
6289,The Hallo-Wiener,Dav Pilkey,childrens-literature,"The story begins with Oscar, a dachshund who ..."
10475,Poor Miss Finch,Wilkie Collins,speculative-fiction,"Twenty-one-year-old Lucilla Finch, the indepe..."
554,Oscar and Lucinda,Peter Carey,historical-fiction fiction novel,"It tells the story of Oscar Hopkins, the Corn..."
11887,Ready Player One,Ernest Cline,science-fiction,The year is 2044 and the world is in near-rui...
...,...,...,...,...
8407,Spinneret,Timothy Zahn,science-fiction speculative-fiction,"The novel is set on year 2016 Earth, with sev..."
3608,Cryptid Hunters,Roland Smith,fantasy,Marty and Grace O'Hara were very different tw...
8614,And So to Murder,John Dickson Carr,mystery detective-fiction novel,"Monica Stanton, the pretty and rather naive d..."
11828,You Don't Know Me,David Klass,young-adult-literature bildungsroman,John is a young fourteen year old boy. He liv...


## Combine Both Recsys Results


In [15]:
result = pd.merge(left, right, on="title", how="left")
result

,title,author_x,genres_x,summary_x,author_y,genres_y,summary_y
0,The Wanderer,Fritz Leiber,science-fiction speculative-fiction fantasy fi...,The novel is set in a future a few decades ah...,NaN,NaN,NaN
1,Glory Road,Robert A. Heinlein,science-fiction speculative-fiction fantasy fi...,"Evelyn Cyril ""E.C."" Gordon (also known as ""Ea...",NaN,NaN,NaN
2,Restoree,Anne McCaffrey,science-fiction speculative-fiction fantasy fi...,"Restoree is the story of Sara, an introverted...",NaN,NaN,NaN
3,Stardust,Neil Gaiman,science-fiction speculative-fiction fantasy fi...,The Faerie Market is held every nine years on...,NaN,NaN,NaN
4,The Gods Themselves,Isaac Asimov,science-fiction speculative-fiction fantasy fi...,The main plotline is a project by aliens who ...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
95,Iron Fist,Aaron Allston,science-fiction speculative-fiction fantasy fi...,After Wraith Squadron's first successful miss...,NaN,NaN,NaN
96,Solo Command,Aaron Allston,science-fiction speculative-fiction fantasy fi...,Wraith Squadron is once again tasked with des...,NaN,NaN,NaN
97,The Hallowed Hunt,Lois McMaster Bujold,science-fiction speculative-fiction fantasy fi...,"The principal characters are: Ingrey, who rec...",NaN,NaN,NaN
98,Behold the Man,Michael Moorcock,science-fiction speculative-fiction fantasy fi...,The story begins with Karl's violent arrival ...,NaN,NaN,NaN


In [16]:
def get_recs(title, df=df):
    l_titles = get_recs_genres(title=title)
    r_titles = get_recs_desc(title=title)
    res = pd.merge(l_titles["title"], r_titles["title"], on="title", how="left")
    res.drop(res[(res["title"] == title)].index, inplace=True)
    res = pd.merge(res, df, on="title", how="outer")
    # res.drop(labels=["genres", "summary"], axis=1, inplace=True)
    return res[0:10]

## Example Usage


In [17]:
get_recs("The Gunslinger")

,title,author,genres,summary
0,The Drawing of the Three,Stephen King,science-fiction childrens-literature horror we...,The book begins less than seven hours after t...
1,Wolves of the Calla,Stephen King,science-fiction horror western-fiction specula...,After escaping the alternate Topeka and the e...
2,Song of Susannah,Stephen King,science-fiction horror western-fiction specula...,Taking place mainly in our world (New York Ci...
3,The Virginian,Owen Wister,western-fiction western childrens-literature f...,The novel begins with an unnamed narrator's a...
4,The Dark Tower: The Wind Through the Keyhole,Stephen King,western-fiction science-fiction fantasy,The novel begins with Roland and his ka-tet a...
5,The Brave Cowboy,Edward Abbey,western-fiction fiction,This book is the story of a cowboy (Jack Burn...
6,Comanche Moon,Larry McMurtry,western childrens-literature fiction,Texas Governor Elisha Pease sends a small tro...
7,Shane,Jack Schaefer,western childrens-literature fiction,The story is set 19th century Wyoming when it...
8,The First Four Years,Laura Ingalls Wilder,western childrens-literature fiction,The First Four Years derives its title from a...
9,Little House in the Big Woods,Laura Ingalls Wilder,western childrens-literature fiction,Little House in the Big Woods describes the h...


In [18]:
get_recs("Use of Weapons")

,title,author,genres,summary
0,Blade Runner 3: Replicant Night,K. W. Jeter,science-fiction speculative-fiction,"Living on Mars, Deckard is acting as a consul..."
1,Blade Runner 2: The Edge of Human,K. W. Jeter,science-fiction speculative-fiction,Beginning several months after the events in ...
2,Icehenge,Kim Stanley Robinson,science-fiction speculative-fiction,"Icehenge is part mystery, part psychological ..."
3,The Fountains of Paradise,Arthur C. Clarke,science-fiction speculative-fiction,"In the 22nd century, Dr Vannevar Morgan is a ..."
4,2010: Odyssey Two,Arthur C. Clarke,science-fiction speculative-fiction,The story is set nine years after the failure...
5,To Sail Beyond the Sunset,Robert A. Heinlein,science-fiction speculative-fiction,The book is a memoir of Maureen Johnson Smith...
6,The Cat Who Walks Through Walls,Robert A. Heinlein,science-fiction speculative-fiction,A writer seated at the best restaurant of the...
7,Methuselah's Children,Robert A. Heinlein,science-fiction speculative-fiction,"The Howard Families derive from Ira Howard, w..."
8,Consider Phlebas,Iain Banks,science-fiction speculative-fiction,The Culture and the Idiran Empire are at war ...
9,Inversions,Iain Banks,science-fiction speculative-fiction,The book takes place on a fictional planet ba...


In [19]:
get_recs("Use of Weapons").to_json()

'{"title":{"0":"Blade Runner 3: Replicant Night","1":"Blade Runner 2: The Edge of Human","2":"Icehenge","3":"The Fountains of Paradise","4":"2010: Odyssey Two","5":"To Sail Beyond the Sunset","6":"The Cat Who Walks Through Walls","7":"Methuselah\'s Children","8":"Consider Phlebas","9":"Inversions"},"author":{"0":"K. W. Jeter","1":"K. W. Jeter","2":"Kim Stanley Robinson","3":"Arthur C. Clarke","4":"Arthur C. Clarke","5":"Robert A. Heinlein","6":"Robert A. Heinlein","7":"Robert A. Heinlein","8":"Iain Banks","9":"Iain Banks"},"genres":{"0":"science-fiction speculative-fiction","1":"science-fiction speculative-fiction","2":"science-fiction speculative-fiction","3":"science-fiction speculative-fiction","4":"science-fiction speculative-fiction","5":"science-fiction speculative-fiction","6":"science-fiction speculative-fiction","7":"science-fiction speculative-fiction","8":"science-fiction speculative-fiction","9":"science-fiction speculative-fiction"},"summary":{"0":" Living on Mars, Deckard

---

Author: Soumyajit Kolay
